CARND - Traffic Sign Classification Project
==================================
German traffic sign classification  as a part of the Self-driving Car Nanodegree program at Udacity using LeNet architecture.

-----------
### Step 0: Load the data

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
import pandas as pd
import random
import numpy as np
import tensorflow as tf
import math

from src import loading, utils, lenet, preprocessing, augmentation
from src.lenet import HYPER_PARAMETERS
logger = utils.get_logger('Main Notebook')

In [ ]:
training, validation, test = loading.load_all()

-------
### Step 1: Dataset Summary & Exploration

Task: provide a basic summary of the dataset, including an exploratory visualisation of the data set.

In [ ]:
summary = utils.get_summary([training, validation, test])
print(summary)

In [ ]:
plt.subplot(211)
plt.hist(training.y, bins=summary['total-no-of-classes']);
plt.title('Training labels distribution');

plt.subplot(212)
plt.hist(validation.y, bins=summary['total-no-of-classes']);
plt.title('Validation labels distribution');
plt.tight_layout()
plt.savefig('./docs/images/training-validation-histogram.jpg')

In [ ]:
# Show that training/validation is somehow balanced
y_train_counts = utils.group_labels_by_counts(training)
y_validation_counts = utils.group_labels_by_counts(validation)
plt.plot(y_train_counts['counts']/ y_validation_counts['counts']);
plt.title('Ratio of training to validation label distribution');
plt.xlabel('Label');
plt.ylabel('(training example count)/(validation example count)');
plt.savefig('./docs/images/training-validation-sample-size-ratios.jpg')

In [ ]:
images = []
labels = []
for i in range(50):
    index = random.randint(0, training.count)
    images.append(training.X[index])
    labels.append(utils.to_sign_label(training.y[index]))
utils.plot_and_save(images, labels, './docs/images/25-random-images.jpg', 5)

In [ ]:
example_images = {
    'with bad lighting': [17894, 14044],
    'good': [22842, 4220]
}
images = []
labels = []
for label, indices in example_images.items():
    for i in indices:
        images.append(training.X[i])
        labels.append(utils.to_sign_label(training.y[i]))

utils.plot_and_save(images, labels, 'docs/images/cherry-pick-good-bad-images.jpg', 2)

----

## Step 2: Design and Test a Model Architecture

#### Pre-process and augment dataset

In [ ]:
# List of enabled data augmenters for training data set
TRAINING_DATA_AUGMENTERS = [
    augmentation.GaussianBlurAugmenter(),
    augmentation.AffineTransformAugmenter(),
    augmentation.AffineTransformAugmenter(),
    augmentation.AffineTransformAugmenter(),
    augmentation.AffineTransformAugmenter(),
    augmentation.AffineTransformAugmenter(),
]
d_train = augmentation.Augmenter.apply(training, TRAINING_DATA_AUGMENTERS)

In [ ]:
# List of enabled data pre-processors
PRE_PROCESSORS = [
    preprocessing.GrayScaleConverter(),
    preprocessing.ZNormaliser(),
]

# Perform pre-processing on augmented training and validation data sets
d_train = preprocessing.PreProcessor.apply(d_train, PRE_PROCESSORS)
d_validation = preprocessing.PreProcessor.apply(validation, PRE_PROCESSORS)

In [ ]:
# Apply pre-processing to two of the "bad" images to show the results
BAD_IMAGE_INDEX = [17894, 14044]

bad_x = np.zeros((2, 32, 32, 3))
bad_y = np.zeros(2,)
for i in range(len(BAD_IMAGE_INDEX)):
    bad_x[i,:] = training.X[BAD_IMAGE_INDEX[i]]
    bad_y[i] = training.y[BAD_IMAGE_INDEX[i]]
bad_examples = loading.DataSet('bad_samples', bad_x, bad_y, len(bad_x))

pre_processed = preprocessing.PreProcessor.apply(bad_examples, PRE_PROCESSORS)

images = []
labels = []
for i in range(len(BAD_IMAGE_INDEX)):
    images.append(training.X[BAD_IMAGE_INDEX[i]]);
    labels.append('Original')

    images.append(pre_processed.X[i].squeeze());
    labels.append('Pre-processed')
utils.plot_and_save(images, labels, './docs/images/bad-images-pre-processed.jpg', 2)

#### Start training

In [ ]:
# This has been executed in the Udacity Workspace using GPU's to speed up training
logger.info('Hyper-parameters: %s', HYPER_PARAMETERS)

tf.reset_default_graph()
placeholders, operations = lenet.setup_graph()

saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    logger.info("Training...")
    for i in range(HYPER_PARAMETERS['EPOCHS']):
        lenet.train_one_epoch(d_train, placeholders, operations)

        training_accuracy = lenet.evaluate(d_train, placeholders, operations)
        validation_accuracy = lenet.evaluate(d_validation, placeholders, operations)

        logger.info("EPOCH {} ...".format(i + 1))
        logger.info("Accuracy on training dataset = {:.3f}".format(training_accuracy))
        logger.info("Validation Accuracy = {:.3f}".format(validation_accuracy))
        logger.info("")

    saver.save(sess, './data/model/lenet')
    logger.info("Model saved")



---
## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

### Load the Images Downloaded from the Internet

In [ ]:
FROM_INTERNET_IMAGE_PATHS = [
    "max-50.jpeg",
    "max-70.jpeg",
    "priority-road.jpg",
    "right-of-way-at-next-intersection.jpg",
    "turn-right-ahead.jpg"
]
FROM_INTERNET_LABELS = [2, 4, 12, 11, 33]
from_internet_images = [utils.read_image_for_lenet('./data/traffic-signs-from-internet/' + p) for p in FROM_INTERNET_IMAGE_PATHS]

plt_labels = ['{}: {}'.format(l, utils.to_sign_label(l)) for l in FROM_INTERNET_LABELS]
utils.plot_and_save(from_internet_images, plt_labels, './docs/images/images-from-internet-resized.jpg', 3)

In [ ]:
# Prepare dataset
d_x = np.stack(from_internet_images)
d_y = np.array(FROM_INTERNET_LABELS)

from_internet_dataset = loading.DataSet('From Internet', d_x, d_y, len(d_x))
logger.info(utils.get_summary([from_internet_dataset]))
from_internet_dataset = preprocessing.PreProcessor.apply(from_internet_dataset, PRE_PROCESSORS)

### Predict the Sign Type for Each Image

In [ ]:
logger.info('Hyper-parameters: %s', HYPER_PARAMETERS)

tf.reset_default_graph()
placeholders, operations = lenet.setup_graph()

softmax_logits = tf.nn.softmax(operations.logits)
all_labels = tf.nn.top_k(softmax_logits, k=43)
top_5_labels = tf.nn.top_k(softmax_logits, k=5)

saver2 = tf.train.Saver()
with tf.Session() as sess:
    saver2.restore(sess, "./data/model/lenet")

    accuracy = lenet.evaluate(from_internet_dataset, placeholders, operations)
    logger.info("Accuracy = {:.3f}".format(accuracy))
    
    top_5_results = sess.run(top_5_labels, feed_dict={
        placeholders.x: from_internet_dataset.X,
        placeholders.keep_probability: 1
    })
    all_labels_results = sess.run(all_labels, feed_dict={
        placeholders.x: from_internet_dataset.X,
        placeholders.keep_probability: 1
    })



### Output Top 5 Softmax Probabilities For Each Image Found on the Web

In [ ]:
for i in range(from_internet_dataset.count):
    print('Actual: {}, {}'.format(FROM_INTERNET_LABELS[i], utils.to_sign_label(FROM_INTERNET_LABELS[i])))
    for j in range(top_5_results.indices.shape[1]):
        predicted_label = top_k_results.indices[i, j]
        print("\t[{}] {}: {}".format(predicted_label, utils.to_sign_label(predicted_label), top_5_results.values[i, j]))

In [ ]:
plt.subplots(figsize=(20, 15))
for i in range(from_internet_dataset.count):
    plt.subplot(from_internet_dataset.count, 1,i+1)
    plt.bar(x=all_labels_results.indices[i], height=all_labels_results.values[i])
    predicted_label = all_labels_results.indices[i][0]
    actual_label = FROM_INTERNET_LABELS[i]
    probability = all_labels_results.values[i][0]
    plt.title("Predicted: {} ({}), Actual: {} ({}), Probability: {:.3f}".format(predicted_label, 
                                                                               utils.to_sign_label(predicted_label),
                                                                               actual_label, 
                                                                               utils.to_sign_label(actual_label),
                                                                               probability))

plt.tight_layout()
plt.savefig("docs/images/softmax-probabilities-on-images-from-internet.jpg")


In [ ]:
MAX_80_LABEL = 5
MAX_80_IMG_PATH = './data/traffic-signs-from-internet/max-80.jpg'
max_80_img = utils.read_image_for_lenet(MAX_80_IMG_PATH)
max_80_dataset = loading.DataSet('Max 80', max_80_img.reshape(1, 32, 32, 3), [MAX_80_LABEL], 1)
d_max_80 = preprocessing.PreProcessor.apply(max_80_dataset, PRE_PROCESSORS)

tf.reset_default_graph()
placeholders, operations = lenet.setup_graph()

softmax_logits = tf.nn.softmax(operations.logits)
all_labels = tf.nn.top_k(softmax_logits, k=43)

saver3 = tf.train.Saver()
with tf.Session() as sess:
    saver3.restore(sess, "./data/model/lenet")
    all_labels_results = sess.run(all_labels, feed_dict={
        placeholders.x: d_max_80.X,
        placeholders.keep_probability: 1
    })

plt.subplot(121)
plt.imshow(max_80_img)
plt.axis('off')
a = plt.subplot(122)
a.fi
plt.bar(x=all_labels_results.indices[0], height=all_labels_results.values[0])
predicted_label = all_labels_results.indices[0][0]
probability = all_labels_results.values[0][0]
plt.title("Predicted: {} ({}), Actual: {} ({}), Probability: {:.3f}".format(predicted_label, 
                                                                           utils.to_sign_label(predicted_label),
                                                                           MAX_80_LABEL, 
                                                                           utils.to_sign_label(MAX_80_LABEL),
                                                                           probability));
plt.savefig("docs/images/softmax-probabilities-on-max-80-sign.jpg")

### Visualize network

In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")

---------------
## Test Dataset

In [ ]:
tf.reset_default_graph()
placeholders, operations = lenet.setup_graph()

d_test = preprocessing.PreProcessor.apply(test, PRE_PROCESSORS)

saver3 = tf.train.Saver()
with tf.Session() as sess:
    saver3.restore(sess, "./data/model/lenet")
    accuracy = lenet.evaluate(d_test, placeholders, operations)
    logger.info("Accuracy = {:.3f}".format(accuracy))